## User tutorial 2 : few components model

This tutorial show how to set up and solve a few components model using EESREP. For a full understanding, read the first tutorial before.


This model takes four main components:

-   load : requests a pre-defined amount of energy;
-   cluster : group of N machines providing energy between its minimal and maximal power when turned-on;
-   fuel : provides the energy requested by the cluster;
-   expensive_unit : provides the remaining energy at a higher cost than the cluster.

A "bus" is added to make a "low of the nodes" between the inputs and the outputs.

The structure of the model can be illustrated as follow:

                expensive_unit -->  |
                                    | -->  load
              fuel --> cluster -->  |

####   Imports

In [1]:
import math

import matplotlib.pyplot as plt
import pandas as pd

from eesrep import Eesrep
from eesrep.components.converter import Cluster, Converter
from eesrep.components.sink_source import FatalSink, Sink, Source

####   Create model

Create the Eesrep object and every components.

A fictive component **price_cluster_on** is added to force the cluster to turn on only when needed, but is not necessary to model this system.

In [2]:
model_A = Eesrep(interface="docplex", name="model_A")

fuel_A = Source(name="fuel", p_min = 0., p_max = 50, price=.5)

fs_df_A = pd.DataFrame({"time": list(range(1001)), 
                            "value": [i*100/1000 for i in range(1001)]})

load_A = FatalSink(name="load", 
                        sink_flow = fs_df_A)

model_A.add_component(fuel_A)
model_A.add_component(load_A)

bus_A = model_A.create_bus(bus_name = "bus")
                            
model_A.plug_to_bus(io = fuel_A.power_out, bus = bus_A, is_input = True, factor=1., offset=0.)
model_A.plug_to_bus(io = load_A.power_in, bus = bus_A, is_input = False, factor=1., offset=0.)

In [3]:
model_B = Eesrep(interface="docplex", name="model_B")

fuel_B = Source(name="fuel", p_min = 0., p_max = 200, price=.5)

fs_df_B = pd.DataFrame({"time": list(range(1001)), 
                            "value": [i*100/1000 for i in range(1001)]})

load_B = FatalSink(name="load", 
                        sink_flow = fs_df_B)

model_B.add_component(fuel_B)
model_B.add_component(load_B)

bus_B = model_B.create_bus(bus_name = "bus")
                            
model_B.plug_to_bus(io = fuel_B.power_out, bus = bus_B, is_input = True, factor=1., offset=0.)
model_B.plug_to_bus(io = load_B.power_in, bus = bus_B, is_input = False, factor=1., offset=0.)

In [4]:
model = Eesrep(interface="docplex")

model.add_submodel(model_A)
model.add_submodel(model_B)

transfert_A_B = Converter(name="A_B", efficiency=1., p_min=0., p_max=None)
transfert_B_A = Converter(name="B_A", efficiency=1., p_min=0., p_max=None)

model.add_component(transfert_A_B)
model.add_component(transfert_B_A)

print(model_A.get_bus_list())

model.plug_to_bus(transfert_A_B.power_in, model_A.get_bus_list()["bus"], False, 1., 0.)
model.plug_to_bus(transfert_B_A.power_in, model_B.get_bus_list()["bus"], False, 1., 0.)

model.plug_to_bus(transfert_A_B.power_out, model_A.get_bus_list()["bus"], True, 1., 0.)
model.plug_to_bus(transfert_B_A.power_out, model_B.get_bus_list()["bus"], True, 1., 0.)

KeyError: 'bus'

####   Set up simulation

In this tutorial, we ask EESREP to split the 1000 time steps resolution in 10 * 100 time steps. Each resolution will start 100 steps after the previous, making no overlap between each resolution.

Each resolution is faster, but more are required. A good balance needs to be found between the time range parameters to get a proper result at a good computation time.

    |-- 100 steps --|  Rolling horizon 1
                    |-- 100 steps --|  Rolling horizon 2
                                    |-- 100 steps --|  Rolling horizon 3


In [ ]:
model.define_time_range(time_step = 1., 
                        time_shift = 100, 
                        future_size = 100, 
                        horizon_count = 10)

####   Solve and get results

In [ ]:
model.solve()
results = model.get_results(as_dataframe=False)

In [ ]:
print(results.keys())
print(results["model_A_fuel"].keys())

The cluster provides all of the requested power as it is cheaper than the other source.

We can see that the cluster turns on machines incrementally as the requested load increases.

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(results["model_A_fuel"]["power_out"], label = "model_A_fuel", color="r")
plt.plot(results["model_A_load"]["power_in"], label = "Load", linestyle = "dashed", color="yellow")
plt.legend()
plt.subplot(122)
plt.plot(results["model_B_fuel"]["power_out"], label = "model_B_fuel", color="g")
plt.plot(results["model_B_load"]["power_in"], label = "Load", linestyle = "dashed", color="yellow")
plt.legend()
plt.show()